Algoritmo de Shor
========================
    
El algoritmo de Shor consiste en reducir el problema de factorizar un número *N* al problema de encontrar el orden *r* de un entero *a* respecto a *N*, éste paso sólo se puede hacer con recursos superpolinomiales en la computación clásica con los algoritmos conocidos, pero hay un algoritmo de complejidad polinomial usando una computadora cuántica. 
    
El proceso general es el siguiente:

1. Se obtiene un número aleatorio *a* que cumpla con $a<N$ y $mcd(a,N)=1$, si $mcd(a,N)\ne 1$ entonces a es un divisor de *N* y ya tenemos un factor de los que estamos buscando.

1. Se calcula el orden de *a* respecto a *N*, es decir el número que cumpla $r = min(\{x \in Z | a^x mod N = 1\})$.

1. Si *r* es un número impar se regresa al paso 1, de lo contrario se calcula $d=mcd(a^{r/2}-1,N)$, si $d\ne 1$ entonces *d* es un factor de  N.

Paso 2 y su implementación cuántica
---------------------------------------

El paso de encontrar el orden de *a* respecto a *N* es polinomial en una computadora cuántica de acuerdo al siguiente algoritmo:

1. Crear el siguiente circuito, con el primer y segundo registros de un tamaño *n* que cumpla $N^2 \le 2^n \le 2N^2$, El primer registro inicializado en $|0\rangle ^{\otimes n}$ y el segundo en $|0\rangle ^{\otimes n-1}\otimes|1\rangle$
![circuito](circuito.png "circuito")
1. La salida de la primera compuerta $QFT_{2^n}$ (el punto marcado por **1** en el circuito) es el estado $2^{-n/2}(|0\rangle+|1\rangle)^{\otimes n}\otimes|0\rangle^{\otimes n-1}|1\rangle$.
1. La compuerta marcada por $U_a^x$ hace la transformación $|y\rangle \rightarrow |a^xmodN\rangle$ donde $x$ es el registro de control descodificándolo en binario ($|5\rangle = |1\rangle|0\rangle|1\rangle$), usando:
$$ \frac{1}{\sqrt{2^n}}(|0\rangle + |1\rangle)^{\otimes n}= \frac{1}{\sqrt{2^n}}\sum_{x=0}^{2n-1}|x\rangle $$
$$\Rightarrow  \frac{1}{\sqrt{2^n}}\sum_{x=0}^{2n-1} U_a^x |x\rangle |1\rangle =  \frac{1}{\sqrt{2^n}}\sum_{x=0}^{2n-1}|x\rangle |a^x modN\rangle = \frac{1}{\sqrt{2^n}}\sum_{b=0}^{r-1}\sum_{z=0}^{m_b-1}|zr+b\rangle |a^b modN\rangle $$
con $m_b$ de tal forma que $zr+b \le 2^n-1$ es un valor que depende del actual $b$ , éste estado es el marcado en el diagrama por **2**
1. Si se mide el segundo registro el estado colapsa a algún valor de $b$ y el primer registro queda en el estado periódico, y el periodo del estado es el orden de $a$ respecto a $N$
$$\frac{1}{\sqrt{m_b}}\sum_{z=0}^{m_b -1} |zr+b\rangle$$
Aplicando la compuerta $QFT_{2^n}^{-1}$ al primer registro la salida (**punto 3**) es una superposición de estados $|x\rangle$ de forma que $x/2^n \approx k/r$ para algún valor de $k<r$
1. Usando el algoritmo de fracciones contínuas se desarrolla la fracción $x/2^n$ medida hasta encontrar un convergente $\frac{c_1}{r_1}$que cumpla $$\|\frac{x}{2^n}-\frac{c_1}{r_1}\| \le \frac{1}{2^{\frac{n-1}{2}}}$$
La medición de $x/2^n$ tiene probabilidad $\frac{4}{r\pi^2}$ de arrojar una fracción útil para este paso, hay que repetir desde el paso 1 hasta obtener dos convergentes distintos $\frac{c_1}{r_1}$, $\frac{c_2}{r_2}$.
1. Después se calcula $r = mcm(r_1,r_2)$, si $a^r modN = 1$ entonces $r$ es el orden de $a$ respecto a $N$, de lo contrario se falló y hay que intentar con un nuevo valor de $a$.

Errores en el circuito
--------------------------------
Las imperfecciones y ruido que hay en un circuito se pueden modelar de la forma $\rho_{f} = (1-p)\rho_i + p\chi\rho_i\chi^\dagger$, donde $\rho_i$ es la matriz de densidad inicial,$\rho_f$ es la matriz de densidad después de la evolución, $\chi$ es el operador *error* y $p$ es la probabilidad de que se ocasione el error en la evolución del sistema.

En el siguiente ejemplo se crea un estado $\Psi_0$ y una matriz de densidad $\rho_0$ correspondiente a este estado puro. *N* es el número que se quiere factorizar y *a* el primo relativo que se usará en la función módulo, *pflip* es la probabilidad de que un Q-bit cambie su estado en un canal.

In [1]:
include("ShorCompuertas.jl")
using algoritmoShor
N = 6
a=5
n = int(ceil(2*log2(N)))
baseQ=big(int(big(2)^(2*n)))
Ψ0 = numtoQvector(int(xytoQstateNum(0,1,n)),n)
ρ0 = Ψ0*Ψ0';
QFT = full(generaQFT(n))
QFTinv = full(generaQFTinv(n))
operadorMod = full(generamodN(n,N,a))
flips = full(generaTransformaciones1flip(n))
pflip = 0.001
pflip*=2*n

0.012

Se calcula $\rho_1$ que es la matriz de densidad posterior a una evolución donde puede haber hasta 3 cambios en cada un Q-bit, *flips* contiente todas las transformaciones $\chi$ que se aplicarán, en éste caso representa el error por no poder crear el estado inicial exacto.

In [3]:
ρ1=aplicaError(ρ0,pflip,flips)
ρ1=aplicaError(ρ1,pflip,flips)
ρ1=aplicaError(ρ1,pflip,flips)

4096x4096 sparse matrix with 299 Float64 entries:
	[1   ,    1]  =  0.0002495
	[2   ,    2]  =  4.1625e-8
	[3   ,    3]  =  4.1625e-8
	[4   ,    4]  =  3.47222e-12
	[5   ,    5]  =  4.1625e-8
	[6   ,    6]  =  3.47222e-12
	[7   ,    7]  =  3.47222e-12
	[9   ,    9]  =  4.1625e-8
	[10  ,   10]  =  3.47222e-12
	[11  ,   11]  =  3.47222e-12
	⋮
	[3073, 3073]  =  3.47222e-12
	[3137, 3137]  =  4.1625e-8
	[3138, 3138]  =  3.47222e-12
	[3139, 3139]  =  3.47222e-12
	[3141, 3141]  =  3.47222e-12
	[3145, 3145]  =  3.47222e-12
	[3153, 3153]  =  3.47222e-12
	[3169, 3169]  =  3.47222e-12
	[3265, 3265]  =  3.47222e-12
	[3393, 3393]  =  3.47222e-12
	[3649, 3649]  =  3.47222e-12

Se calcula $\rho_2$ que es el estado posterior a la *QFT* y luego se calcula $\rho_3$ que de nuevo es aplicar el error donde puede haber hasta 3 flips en cada Q-bit

In [4]:
ρ2=QFT*ρ1*QFT'
ρ3=aplicaError(ρ2,pflip,flips)
ρ3=aplicaError(ρ3,pflip,flips)
ρ3=aplicaError(ρ3,pflip,flips)

4096x4096 Array{Complex{Float64},2}:
 7.79299e-6-4.46539e-33im   …                   0.0+0.0im
 7.78748e-6+3.86712e-9im                        0.0+0.0im
 7.78749e-6+3.70806e-9im                        0.0+0.0im
   7.784e-6+2.39605e-9im                        0.0+0.0im
 7.78752e-6+3.39149e-9im                        0.0+0.0im
 7.78219e-6+4.08729e-9im    …                   0.0+0.0im
 7.78407e-6+1.92513e-9im                        0.0+0.0im
 7.78285e-6+4.72173e-10im                       0.0+0.0im
 7.78764e-6+2.77051e-9im                        0.0+0.0im
 7.78199e-6+4.99334e-9im                        0.0+0.0im
  7.78238e-6+3.3227e-9im    …                   0.0+0.0im
 7.77972e-6+8.05637e-10im                       0.0+0.0im
 7.78435e-6+1.02384e-9im                        0.0+0.0im
                    ⋮       ⋱                      ⋮     
                 0.0+0.0im      4.51732e-20-9.3553e-25im 
                 0.0+0.0im  …  4.51763e-20-3.85848e-24im 
                 0.0+0.0im     4.51

se calcula $\rho_4$ que es el estado después de la compuerta controlada $U_a^x$ y de nuevo se aplican los errores y el resultado es $\rho_5$

In [5]:
ρ4 = operadorMod*ρ3*operadorMod'
ρ5 = aplicaError(ρ4,pflip,flips)
ρ5 = aplicaError(ρ5,pflip,flips)
ρ5 = aplicaError(ρ5,pflip,flips)

4096x4096 Array{Complex{Float64},2}:
 1.16713e-5-1.21246e-31im   …                   0.0+0.0im
 7.77581e-6+3.85939e-9im                        0.0+0.0im
 1.16625e-5+5.92054e-9im                        0.0+0.0im
 7.77234e-6+2.39126e-9im                        0.0+0.0im
 1.16626e-5+5.41508e-9im                        0.0+0.0im
 7.77053e-6+4.07912e-9im    …                   0.0+0.0im
 1.16571e-5+3.07372e-9im                        0.0+0.0im
 7.77118e-6+4.71764e-10im                       0.0+0.0im
 1.16628e-5+4.42359e-9im                        0.0+0.0im
 7.77033e-6+4.98336e-9im                        0.0+0.0im
 1.16544e-5+5.30505e-9im    …                   0.0+0.0im
  7.76807e-6+8.0417e-10im                       0.0+0.0im
  1.16575e-5+1.6347e-9im                        0.0+0.0im
                    ⋮       ⋱                      ⋮     
                 0.0+0.0im      9.46678e-19-5.2273e-23im 
                 0.0+0.0im  …   9.4685e-19-2.15578e-22im 
                 0.0+0.0im     9.46

$\rho_6$ es el estado posterior a *QFT*$^{-1}$, de nuevo se altera el estado con las transformaciones de errores. $\rho_f$ es el estado final del circuito, corresponde al estado que se mediría

In [11]:
ρ6 = QFTinv*ρ5*QFTinv'

4096x4096 Array{Complex{Float64},2}:
  0.000559596-2.94395e-26im   …                    0.0+0.0im
 -2.66435e-20-1.97518e-21im                        0.0+0.0im
 -9.75081e-21+3.25035e-21im                        0.0+0.0im
 -4.47739e-20+5.13411e-20im                        0.0+0.0im
 -8.14537e-20-2.50022e-20im                        0.0+0.0im
  -2.83118e-20-6.9473e-20im   …                    0.0+0.0im
 -2.65319e-19-3.59861e-20im                        0.0+0.0im
  1.96363e-19-2.07481e-20im                        0.0+0.0im
 -9.93721e-20-4.20427e-20im                        0.0+0.0im
  5.56432e-20-3.25954e-20im                        0.0+0.0im
 -9.59702e-20-1.08763e-19im   …                    0.0+0.0im
  1.50298e-20-2.06658e-19im                        0.0+0.0im
 -3.79461e-20-6.96634e-20im                        0.0+0.0im
                      ⋮       ⋱                       ⋮     
                   0.0+0.0im      3.90774e-35-1.02478e-35im 
                   0.0+0.0im  …   2.51065e-34-1.

In [12]:
ρf=aplicaError(ρ6,pflip,flips)

elapsed time: 28.036605839 seconds (9935281224 bytes allocated, 6.12% gc time)


4096x4096 Array{Complex{Float64},2}:
  0.000579776-5.78725e-25im   …                    0.0+0.0im
 -2.77792e-20-1.65911e-21im                        0.0+0.0im
 -1.00928e-20+3.92097e-21im                        0.0+0.0im
 -4.88734e-20+5.44093e-20im                        0.0+0.0im
  1.38409e-15+3.10349e-16im                        0.0+0.0im
 -3.05464e-20-7.35879e-20im   …                    0.0+0.0im
 -2.74092e-19-3.84602e-20im                        0.0+0.0im
  2.02269e-19-1.48408e-20im                        0.0+0.0im
   2.2326e-15+1.05962e-15im                        0.0+0.0im
  6.05636e-20-3.77845e-20im                        0.0+0.0im
 -9.89837e-20-1.18602e-19im   …                    0.0+0.0im
   1.4612e-20-2.22259e-19im                        0.0+0.0im
  3.91621e-16+3.10402e-16im                        0.0+0.0im
                      ⋮       ⋱                       ⋮     
                   0.0+0.0im      1.23558e-34+1.21519e-35im 
                   0.0+0.0im  …    2.02721e-33+3

In [13]:
ρf=aplicaError(ρf,pflip,flips)
ρf=aplicaError(ρf,pflip,flips)

4096x4096 Array{Complex{Float64},2}:
  0.000620035-1.53307e-24im   …                    0.0+0.0im
 -3.00627e-20-1.02641e-21im                        0.0+0.0im
  -1.07942e-20+5.2668e-21im                        0.0+0.0im
 -5.70572e-20+6.05453e-20im                        0.0+0.0im
  4.53594e-15+1.09081e-15im                        0.0+0.0im
  -3.5013e-20-8.18063e-20im   …                    0.0+0.0im
 -2.91592e-19-4.33885e-20im                        0.0+0.0im
  2.14055e-19-3.06625e-21im                        0.0+0.0im
  7.23924e-15+3.72431e-15im                        0.0+0.0im
   7.03773e-20-4.8149e-20im                        0.0+0.0im
 -1.05001e-19-1.38212e-19im   …                    0.0+0.0im
  1.37837e-20-2.53376e-19im                        0.0+0.0im
  1.23989e-15+1.09131e-15im                        0.0+0.0im
                      ⋮       ⋱                       ⋮     
                   0.0+0.0im      1.07519e-33+1.19642e-33im 
                   0.0+0.0im  …   2.36302e-32+6.

Se calcula $\Psi_f$, el estado final obtenido con error cero, usando compuertas ideales.

In [14]:
Ψf=QFTinv*operadorMod*QFT*Ψ0

4096x1 Array{Complex{Float64},2}:
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
    ⋮     
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im

Con el estado final ideal, y el estado final con estimación de errores se calcula la probabilidad de que un circuito con esos errores genere el mismo resultado que un circuito ideal. Para el caso de un circuito con dos registros de 6 bits, con una aplicación de errores con $pflip=0.001$ entre cada compuerta la probabilidad calculada es de 0.999692, con tres aplicaciones de errores la probabilidad es de 0.890.

In [15]:
probEstadoDeseado = Ψf'*ρf*Ψf

1x1 Array{Complex{Float64},2}:
 0.990791+2.24373e-22im